# Load dataset

In [ ]:
import json

with open('companies_cross.json', 'r') as f:
    companies = json.load(f)

In [ ]:
from collections import Counter

# Let's use only companies that we currently have website of (at least some pages)
companies = [c for c in companies if c.get('path_to')]

# Let's also limit the number of sites (because this is not cleaned yet)
companies = [c for c in companies if c.get('current_site_count', 0) < 100]

# Let's use only well represented industries
industries = Counter([c['industry'] for c in companies])
companies = [c for c in companies if industries[c.get('industry')] > 500]

## Some helper functions

In [ ]:
import os
import gzip

from scandir import walk
from bs4 import BeautifulSoup

from ipyparallel import require

import logging

# set root logger level
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)

# setup custom logger
logger = logging.getLogger(__name__)
handler = logging.FileHandler('extract_features.log')
handler.setLevel(logging.INFO)
logger.addHandler(handler)


@require(BeautifulSoup)
def get_soup_from_html_gz(path):
    with gzip.open(path, 'rt') as f:  # TODO Apparently, not all files are utf-8 encoded? :o
        soup = BeautifulSoup(f, 'html.parser')
    return soup


@require('os', save_texts_for_domain)
def get_texts_for_domain(path, get_text, force_read=False, file_name=None):
    file_name = file_name or 'full_texts.txt'
    text_file = os.path.join(path, file_name)
    if not os.path.exists(text_file) or force_read:
        text = get_text(path)
        text_file = save_texts_for_domain(text, path, file_name=file_name)
    else:
        with open(text_file, 'r') as f:
            text = f.read()
    return text, text_file


@require(get_soup_from_html_gz, sites_for_domain)
def get_full_text(path):
    text = ''
    for site_path in sites_for_domain(path):
        body = get_soup_from_html_gz(site_path).body
        if body:
            text += body.get_text()
            text += '\n'
    return text


@require(get_soup_from_html_gz, sites_for_domain)
def get_nav_text(path):
    text = ''
    for site_path in sites_for_domain(path):
        soup = get_soup_from_html_gz(site_path)
        text = get_nav_menu(soup)
        if text:
            text = ' '.join(text)
            break
        else:
            text = ''
    return text


@require('os')
def save_texts_for_domain(text, path, file_name=None):
    file_name = file_name or 'full_texts.txt'
    file_name = os.path.join(path, file_name)
    with open(file_name, 'w') as f:
        f.write(text)
    return file_name


@require(walk, 'os')
def sites_for_domain(path):
    for dirpath, dirs, file_names in walk(path):
        for file_name in file_names:
            if file_name.endswith('.gz'):
                yield os.path.join(dirpath, file_name)

                
def get_nav_menu(soup):
    if not soup.body:
        return None
    # We expect nav menu in header
    # First look for possible headers
    header = soup.body.find_all('header')
    if not header:
        header = soup.body.find_all({'class': ['header', 'menu']})
    if not header:
        header = soup.body.find_all(id='header')
    
    # Then look for nav menu in that header
    if header:
        header = header[0]
        has_header = True
    # If not found, look for a menu anywhere
    else:
        has_header = False
        header = soup.body
        
    nav = header.find('nav')
    if not nav:
        nav = soup.body.find('nav')
    if not nav:
        nav = header.find(class_='nav')
    if not nav:
        nav = header.find(id='nav')
    
    if nav:
        return [item for li in nav.find_all('li') for item in li.stripped_strings]  # TODO this doesn't deal with submenus
    elif has_header:
        return [item for li in header.find_all('li') for item in li.stripped_strings]
    else:
        return []

    

# Detect website language

In [ ]:
from ipyparallel import Client


#  Let's parallelize this
c = Client()
dv = c[:]
dv.block = False
dv
lview = c.load_balanced_view()

In [ ]:
from tqdm import tqdm, tqdm_notebook
from langdetect import detect
from ipyparallel import require


@require(get_texts_for_domain, get_full_text, 'langdetect')
def get_website_language(company):
    try:
        path = company['path_to']
        website, file_path = get_texts_for_domain(path, get_full_text)
        language = langdetect.detect(website)
    except Exception as e:
#         logger.debug(str(e))
        print(e)

    return language

results = {}
for i, company in enumerate(tqdm(companies)):
#     if not company.get('website_lang'):
    ar = lview.apply_async(get_website_language, company)
    results[i] = ar
    

In [ ]:
for i, company in enumerate(tqdm(companies)):
    try:
        language = results.get(i).get()
    except Exception as e:
        language = None
    if language:
        company['website_lang'] = language
    else:
        company['website_lang'] = 'NA'

In [ ]:
ar = lview.apply_async(get_website_language, companies[0])

## Save dataset

In [ ]:
import json

with open('companies_cross.json', 'w') as f:
    json.dump(companies, f)

# Select only english websites

In [ ]:
companies = [c for c in companies if c.get('website_lang') == 'en']

# Get bag of words for websites

In [ ]:
from tqdm import tqdm, tqdm_notebook
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.utils import shuffle


# Let's shuffle the data first
companies = shuffle(companies)
train_data = companies[:100000]
target = []

texts = []
hv = HashingVectorizer(non_negative=True, input='filename', stop_words='english', ngram_range=(1,2))
for company in tqdm(train_data):
    path = company['path_to']
    try:
        text, file_path = get_texts_for_domain(path, get_full_text, force_read=True)  # TODO remove force_read
    
        texts.append(file_path)
        target.append(company['industry'])
    except Exception as e:
        print(e)
        logger.error(str(e))

try:
    bag = hv.transform(texts)
    del texts
    tf_transformer = TfidfTransformer()
    bag_train_tf = tf_transformer.fit_transform(bag)
except Exception as e:
    print(e)
    logger.error(str(e))

# Accuracy about 43%
# English only - 48%
# English only + stop words + ngrams - 49%


In [ ]:
tf_transformer = TfidfTransformer()
bag_train_tf = tf_transformer.fit_transform(bag)

## Get bag of words on nav menus

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.utils import shuffle


# Let's shuffle the data first
companies = shuffle(companies)
train_data = companies[:160000]
target = []

texts = []
hv = HashingVectorizer(non_negative=True, input='filename')
for company in tqdm(train_data):
    text = ''
    path = company['path_to']
    try:
        text, file_path = get_texts_for_domain(path, get_nav_text)
        
        if text:
            target.append(company['industry'])
            texts.append(file_path)
    except Exception as e:
        print(e)
        logger.error(str(e))

bag = hv.transform(texts)
del texts

tf_transformer = TfidfTransformer()
bag_train_tf = tf_transformer.fit_transform(bag)

# Accuracy about 61%


In [ ]:
# Try SGD Classifier
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', n_iter=100, alpha=0.01, n_jobs=4)
clf.fit(bag_train_tf, target)

In [ ]:
clf

In [ ]:
# Try naive bayes classifier
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(bag_train_tf, target)

In [ ]:
texts = []
# Test on 1000 samples
test_companies = companies[-2000:]
tested = []
for company in tqdm(test_companies):
    text = ''
    path = company['path_to']
    try:
        text, file_path = get_texts_for_domain(path, get_full_text)

        texts.append(file_path)
        tested.append(company)
    except Exception as e:
        print(e)
bag = hv.transform(texts)
del texts

bag_test_tf = tf_transformer.transform(bag)

predicted = clf.predict(bag_test_tf)
score = clf.score(bag_test_tf, [c['industry'] for c in tested])

In [ ]:
score

In [ ]:
correct = 0
similar = {}
for company, predict in zip(tested, predicted):
    if company['industry'] == predict or company['industry'] in similar.get(predict, []) or predict in similar.get(company['industry'], []):
        correct += 1
    else:
        print('%s - predicted %s' % (company['industry'], predict))
#         add = input('Add? yN ')
        add = 'n'
        if add == 'y':
            s = similar.get(company['industry'], [])
            s.append(predict)
            similar[company['industry']] = s
            
            s = similar.get(predict, [])
            s.append(company['industry'])
            similar[predict] = s
            correct += 1

correct